In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
url.content
soup = BeautifulSoup(url.content, 'html5lib')


In [3]:
table_contents=[]
table=soup.find('table')
for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [5]:
df.groupby(['PostalCode']).first()

Borough  \
PostalCode                        
M1B                 Scarborough   
M1C                 Scarborough   
M1E                 Scarborough   
M1G                 Scarborough   
M1H                 Scarborough   
...                         ...   
M9N                        York   
M9P                   Etobicoke   
M9R                   Etobicoke   
M9V                   Etobicoke   
M9W         Etobicoke Northwest   

                                                 Neighborhood  
PostalCode                                                     
M1B                                            Malvern, Rouge  
M1C                    Rouge Hill, Port Union, Highland Creek  
M1E                         Guildwood, Morningside, West Hill  
M1G                                                    Woburn  
M1H                                                 Cedarbrae  
...                                                       ...  
M9N                                                    Weston  
M9P                                                 Westmount  
M9R         Kingsview Village, St. Phillips, Martin Grove ...  
M9V         South Steeles, Silverstone, Humbergate, Jamest...  
M9W         Clairville, Humberwood, Woodbine Downs, West H...  

[103 rows x 2 columns]

# Part 2

In [6]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\sumchat\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [7]:
import geocoder

In [8]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

In [9]:
df_geo = pd.read_csv(url)
df_geo.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [10]:
df_geo.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [11]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [12]:
df.shape

(103, 3)

In [13]:
df_geo.shape

(103, 3)

In [14]:
df = df.join(df_geo.set_index('Postal Code'), on='PostalCode')
df

PostalCode                Borough  \
0          M3A             North York   
1          M4A             North York   
2          M5A       Downtown Toronto   
3          M6A             North York   
4          M7A           Queen's Park   
..         ...                    ...   
98         M8X              Etobicoke   
99         M4Y       Downtown Toronto   
100        M7Y  East Toronto Business   
101        M8Y              Etobicoke   
102        M8Z              Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4                        Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100                                     Enclave of M4L  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 5 columns]